In [6]:
def total_transaction_fee(signal, transaction_amount) -> float:
    '''
    @params:
        signal:             1 for buy; -1 for sell
        transaction_amount: transaction price times shares
    @return:
        total transaction cost
    '''

    if transaction_amount == 0:
        return 0
    commision_charge = max(transaction_amount * 0.003, 5)
    transfer_fee = transaction_amount * 0.00002
    stamp_tax = transaction_amount * 0.001

    return commision_charge + transfer_fee + stamp_tax if signal == -1 else commision_charge + transfer_fee

In [7]:
import yfinance as yf
import pandas_ta as ta
import pandas as pd
import numpy as np

df = yf.Ticker("600887.SS").history(start='2020-08-01', end='2021-11-01')[['Open','High','Low', 'Close']]
# process data
df.ta.rsi(close='Close', length=14, append=True, signal_indicators=True, xa=70, xb=30)
df.columns = [x.lower() for x in df.columns]
test_data = df[df.index >= '2020-11-01']

initial_balance = 1000000
initial_shares = 0

cash_balance = initial_balance
share_holding = initial_shares
yesterday_flag = 0
trading_outlay = list()

trading_results = pd.DataFrame(columns=["date","signal","cash_balance", "share_holding", "asset"
                                        , "transaction_price", "transaction_cost", "trading_outlay", "reward" ])

for row in test_data.iterrows():

    reward = 0
    transaction_price = (row[1].high+row[1].low)/2
    transaction_cost = 0

    if yesterday_flag == 1: #buy
        # the logic here is to assume I pay the highest fees, how many hands can I afford
        max_hand = np.floor((cash_balance)/(transaction_price*1.00302)/100)
        transaction_shares = max_hand * 100
        
        transaction_amount = transaction_price*transaction_shares
        transaction_cost = total_transaction_fee(yesterday_flag, transaction_amount)
        
        cash_balance = cash_balance-transaction_amount-transaction_cost
        share_holding = share_holding+transaction_shares
        
        trading_outlay.append(transaction_cost+transaction_amount)
        
    elif yesterday_flag == -1: #sell
        transaction_shares = share_holding
        transaction_amount = transaction_price*transaction_shares
        transaction_cost = total_transaction_fee(yesterday_flag, transaction_amount)
        
        cash_balance = cash_balance+transaction_amount-transaction_cost
        share_holding = 0
        reward = transaction_amount-transaction_cost-np.sum(trading_outlay)
        trading_outlay = list()

    
    asset = round(cash_balance+share_holding*transaction_price,0)
#     print(trading_outlay)
    trading_results = trading_results.append({"date":row[0],"signal":yesterday_flag,"cash_balance":round(cash_balance,2)
                                              , "share_holding":share_holding, "asset":asset
                                              ,"transaction_price":transaction_price, "transaction_cost":transaction_cost
                                              ,"trading_outlay": np.sum(trading_outlay),"reward":reward
                                             }, ignore_index=True)

    # observe today's trading signal and carry out tomorrow
    if row[1].rsi_14_b_30 == 1:
        yesterday_flag = 1
    elif row[1].rsi_14_a_70 == 1:
        yesterday_flag = -1
    else:
        yesterday_flag = 0
        
trading_results.to_excel("trading_results.xlsx")

In [15]:
test_data

,open,high,low,close,rsi_14,rsi_14_a_70,rsi_14_b_30
Date,,,,,,,
2020-11-02,37.894773,38.531247,36.572860,37.111420,40.964665,0,0
2020-11-03,37.287674,38.501871,37.140794,38.296242,45.645443,0,0
2020-11-04,38.384369,38.844588,37.610806,38.638958,46.955566,0,0
2020-11-05,38.922925,39.118765,38.100403,38.658543,47.034132,0,0
2020-11-06,39.011054,39.020844,37.777272,38.071030,44.886310,0,0
...,...,...,...,...,...,...,...
2021-10-25,40.000000,41.180000,39.849998,40.500000,69.244393,0,0
2021-10-26,40.500000,40.990002,40.000000,40.560001,69.504768,0,0
2021-10-27,40.400002,40.639999,39.720001,40.279999,66.668290,0,0


In [28]:
# Import necessary libraries
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Create Figure
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, row_width=[0.25, 0.75])
# Inspect Result
# print(fig)

# Create Candlestick chart for price data
fig.add_trace(go.Candlestick(
    x=test_data.index,
    open=test_data['open'],
    high=test_data['high'],
    low=test_data['low'],
    close=test_data['close'],
    increasing_line_color='#ef476f',
    decreasing_line_color='#06d6a0',
    showlegend=False
), row=1, col=1)


# plot buying signal
buying_txt = trading_results[(trading_results.signal == 1) & (trading_results.transaction_cost>0)]
buying_signal = trading_results[(trading_results.signal == 1)]

fig.append_trace(
    go.Scatter(
        x=buying_signal.date,
        y=buying_signal.transaction_price*0.96,
        marker = dict(
            color='#073b4c',
            size=10,
            line=dict(
                color='#118ab2',
                width=2
            ),
            symbol='triangle-up'
            
        ),
#         mode = "markers+text",
        mode = "markers",
        name = "buy",
        text = buying_txt.transaction_price,
        textposition="bottom center",
        texttemplate='%{text:$.2f}',
        
    ), row=1, col=1
)

fig.append_trace(
    go.Scatter(
        x=buying_txt.date,
        y=buying_txt.transaction_price*0.96,
        marker = dict(
            color='#073b4c',
            size=10,
            line=dict(
                color='#118ab2',
                width=2
            ),
            symbol='triangle-up'
            
        ),
        mode = "markers+text",
#         name = "buy",
        text = buying_txt.transaction_price,
        textposition="bottom center",
        texttemplate='%{text:$.2f}',
        showlegend=False,
    ), row=1, col=1
)


# plot selling signal
# sell_signal = trading_results[(trading_results.signal == -1) & (trading_results.transaction_cost>0)]
sell_signal = trading_results[(trading_results.signal == -1)]
sell_txt = trading_results[(trading_results.signal == -1) & (trading_results.transaction_cost>0)]

fig.append_trace(
    go.Scatter(
        x=sell_signal.date,
        y=sell_signal.transaction_price*1.04,
        marker = dict(
            color='#fb5607',
            size=10,
            line=dict(
                color='#ffbe0b',
                width=2
            ),
            symbol='triangle-down'
            
        ),
#         mode = "markers+text",
        mode = "markers",
        name = "sell",
        text = sell_txt.transaction_price,
        textposition="top center",
        texttemplate='%{text:$.2f}',
#         textsize = 1
    ), row=1, col=1
)

fig.append_trace(
    go.Scatter(
        x=sell_txt.date,
        y=sell_txt.transaction_price*1.04,
        marker = dict(
            color='#fb5607',
            size=10,
            line=dict(
                color='#ffbe0b',
                width=2
            ),
            symbol='triangle-down'
            
        ),
        mode = "markers+text",
        name = "sell",
        text = sell_txt.transaction_price,
        textposition="top center",
        texttemplate='%{text:$.2f}',
        showlegend=False,
    ), row=1, col=1
)



# Make RSI Plot
fig.add_trace(go.Scatter(
    x=test_data.index,
    y=test_data['rsi_14'],
    line=dict(color='#ffd166', width=2),
#     showlegend=False,
    name="RSI"
), row=2, col=1
)

# Add upper/lower bounds
fig.update_yaxes(range=[-10, 110], row=2, col=1)
fig.add_hline(y=0, col=1, row=2, line_color="#666", line_width=2)
fig.add_hline(y=100, col=1, row=2, line_color="#666", line_width=2)

# Add overbought/oversold
fig.add_hline(y=30, col=1, row=2, line_color='#336699', line_width=2, line_dash='dash')
fig.add_hline(y=70, col=1, row=2, line_color='#336699', line_width=2, line_dash='dash')

# Customize font, colors, hide range slider
# make the figure prettier
layout = go.Layout(
    plot_bgcolor='#ecf8f8',
    font_family='Monospace',
    font_color='#073b4c',
    font_size=10,
    xaxis=dict(
        rangeslider=dict(visible=False)
    ),
    autosize=False,
    width=1000,
    height=400,
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0,
        pad=4
    )
)

fig.update_xaxes(
    rangebreaks=[
        dict(bounds=['sat', 'mon'])
    ]
)
# update and display
fig.update_layout(layout)
fig.show()